In [ ]:
from symusic import Score
import pandas as pd
import numpy as np
import json
import os
import sys
import math
import torch
import random
from matplotlib import pyplot as plt
from utils import track_to_list

from data.Dataset import MidiDataset
from torch.utils.data import DataLoader

In [ ]:
dataset = MidiDataset(json_dir='./data/data.json', max_seq_len=512)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
full_dataset = MidiDataset('./data.json', 512)
train_dataset = MidiDataset('./data.json', 512, 'train', .7)
test_dataset = MidiDataset('./data.json', 512, 'test', .7)

In [ ]:
iter(dataloader)

### Notes:
Only work on single time_signature, key_signarture, or tempo tracks for now.

## Dataset

In [ ]:
def get_song_details(dir):
    """
    Input midi directory.
    Output song details as a dict().
    """
    try:
        midi = Score(dir)
    except:
        print(f'ERROR with: {dir}')
        return None

    n_tracks = sum(1 if not t.is_drum else 0 for t in midi.tracks) # Count non-drum tracks.
    if n_tracks <= 1:                   # Discard songs with only one track.
        print(f'SINGLE TRACK: {dir}')
        return None

    time_signatures = midi.time_signatures
    if len(time_signatures) > 1:        # Discard songs with multiple time signatures.
        print(f'MULTI SIGNATURE: {dir}')
        return None

    tempos = midi.tempos
    if len(tempos) > 1:                 # Discard songs with multiple tempos.
        print(f'MULTI TEMPO: {dir}')
        return None

    keys = midi.key_signatures
    if len(keys) > 1:                   # Discard songs with multiple keys.
        print(f'MULTI KEY: {dir}')
        return None

    song = {
        'Midi_Directory': dir,
        'Artist': dir.split('/')[-2],
        'Song': dir.split('/')[-1][:-4]
    }
    return song

# def is_synchronous(midi):
    

In [ ]:
# data = []
# data_folder = './archive'

# for root, dirs, files in os.walk(data_folder):
#     for file in files:
#         if not file.endswith('.mid'):
#             continue
#         midi_directory = os.path.join(root, file)
#         song_details = get_song_details(midi_directory)
#         if song_details:
#             data.append(song_details)

# print(f"Number of songs: {len(data)}")
# json_file_path = "data.json"
# with open(json_file_path, "w") as json_file:
#     json.dump(data, json_file, indent=4)

In [ ]:
midi = Score('./archive/VOF_de_Kunst/Een_kopje_koffie.mid')

In [ ]:
track = midi.tracks[14]
print(f"name: {track.name}")


frame_pianoroll = track.pianoroll(['frame'])
print("frame_pianoroll", frame_pianoroll.dtype, frame_pianoroll.shape)
plt.imshow(frame_pianoroll[0], aspect='auto')

In [ ]:
def get_clipped_tracks(midi, n_bars=16, zero_index=True, include_empty_start=False, visualize=False, debug=False):
    '''
    Returns smaller clips in the size "n_bars".
    '''
    time_sig = (midi.time_signatures[0].numerator, midi.time_signatures[0].denominator)
    bar_length = int(midi.tpq * time_sig[0] * (4 / time_sig[1]))
    first_note_time = min(t.notes[0].time for t in midi.tracks if t.notes)

    note_32nd = midi.tpq // 8

    if include_empty_start:
        loop_start = midi.start()
    else:
        for s in range(midi.start(), midi.end(), bar_length):
            if first_note_time <= s:
                loop_start = s
                break

    clipped_tracks = dict()

    for track in midi.tracks:

        if track.is_drum:   # Skip if drum track.
            continue

        if debug or visualize:
            print(f"Processing track \"{track.name}\"")

        clipped_track = []

        for i in range(loop_start, midi.end(), n_bars * bar_length):
            start_tick = i - note_32nd
            end_tick = min(start_tick + (n_bars * bar_length), midi.end())
            
            if end_tick - start_tick < bar_length:  # Skip if less than one bar.
                continue
            
            clipped_part = track.clip(start_tick, end_tick, clip_end=True)
            if zero_index:
                notes_list = track_to_list(clipped_part, start_tick)
            else:
                notes_list = track_to_list(clipped_part)
            clipped_track.append(notes_list)

            if debug:
                print("---")
                print(f"Clip from: {i}")
                print(f"Bars in clip: {(end_tick - start_tick) / bar_length}")
                print(f"Number of notes in clip: {len(clipped_part.notes)}")

            if visualize:
                frame_pianoroll = clipped_part.pianoroll(['frame'])
                plt.imshow(frame_pianoroll[0,:,start_tick:], aspect='auto')
                plt.title(f"Clip from: {i}")
                plt.show()
        
        clipped_tracks[track.name] = clipped_track

    return clipped_tracks

In [ ]:
clipped_midi = get_clipped_tracks(midi, n_bars=4)

In [ ]:
clipped_midi.keys()

In [ ]:
clipped_midi

In [ ]:
dic = {
    'a': 1,
    'b': 2
}

list(dic.items())

In [ ]:
from datetime import datetime
print(f"log {datetime.now()}.txt")

In [ ]:
# random.seed = 42
with open('./data.json', 'r') as data_json:
    data = json.load(data_json)
random.Random(42).shuffle(data)

In [ ]:
data

In [ ]:
from datetime import datetime, date
print(date.today())
print(datetime.now().date())

In [ ]:
datetime.now()